In [ ]:
using OrdinaryDiffEq
using BenchmarkTools

function test_parallel(t, u, du)
    #@boundscheck @assert size(u) == size(du)
    @inbounds Threads.@threads for i in 1:length(u)
        du[i] = exp(u[i])
    end
    nothing
end

function test_serial(t, u, du)
    #@boundscheck @assert size(u) == size(du)
    @inbounds for i in eachindex(u)
        du[i] = exp(u[i])
    end
    nothing
end

function display_benchmarks(u0, t)
    u = copy(u0)
    du = similar(u0)
    display("serial")
    display(@benchmark test_serial(t, u, du))
    display("parallel")
    display(@benchmark test_parallel(t, u, du))
    nothing
end

function time_solve(u0, tspan, dt)
    ode_serial = ODEProblem(test_serial, u0, tspan)
    ode_parallel = ODEProblem(test_parallel, u0, tspan)
    println("serial")
    @time solve(ode_serial, SSPRK33(), dt=dt, save_everystep=false)
    println("parallel")
    @time solve(ode_parallel, SSPRK33(), dt=dt, save_everystep=false)
    nothing
end

srand(12345)
u0 = zeros(10^4)
tspan = (0., 10.)
dt = 1.e-3

display_benchmarks(u0, tspan[1])
time_solve(u0, tspan, dt)


# Benchmark

In [ ]:
using OrdinaryDiffEq
import HyperbolicDiffEq; #reload("HyperbolicDiffEq")

HCL=HyperbolicDiffEq
LobattoLegendre = HCL.LobattoLegendre; UniformPeriodicMesh1D = HCL.UniformPeriodicMesh1D
compute_coefficients = HCL.compute_coefficients; evaluate_coefficients = HCL.evaluate_coefficients
semidiscretise = HCL.semidiscretise; UniformPeriodicFluxDiffDisc2D = HCL.UniformPeriodicFluxDiffDisc2D
EnergyConservativeFlux = HCL.EnergyConservativeFlux; GodunovFLux = HCL.godunov
EulerVar2D = HCL.EulerVar2D; conserved_variables = HCL.conserved_variables; SuliciuFlux=HCL.SuliciuFlux
LocalLaxFriedrichsFlux = HCL.LocalLaxFriedrichsFlux; ChandrashekarFluxEC=HCL.ChandrashekarFluxEC
IsmailRoeFluxEC=HCL.IsmailRoeFluxEC; CentralFlux=HCL.CentralFlux; RanochaFluxECandKEP=HCL.RanochaFluxECandKEP
MorinishiFlux=HCL.MorinishiFlux; DucrosEtAlFlux=HCL.DucrosEtAlFlux; KennedyGruberFlux=HCL.KennedyGruberFlux
PirozzoliFlux=HCL.PirozzoliFlux
integrate=HCL.integrate; kinetic_energy=HCL.kinetic_energy; entropy=HCL.entropy

balance_law = HyperbolicDiffEq.Euler{Float64,2}(1.4)

ϱ₀(x,y)  = 1.
vx₀(x,y) = sin(x)*cos(y)
vy₀(x,y) = -cos(x)*sin(y)
p₀(x,y)  = 100/γ + (cos(2x)+cos(2y))*3/16

u₀(x,y) = conserved_variables(ϱ₀(x,y), vx₀(x,y), vy₀(x,y), p₀(x,y), balance_law)
xmin=0.; xmax=2π; ymin=0.; ymax=2π

#fvol = CentralFlux()
#fvol = MorinishiFlux()
#fvol = DucrosEtAlFlux()
#fvol = KennedyGruberFlux()
fvol = PirozzoliFlux()
#fvol = EnergyConservativeFlux()
#fvol = ChandrashekarFluxEC()
#fvol = IsmailRoeFluxEC()
#fvol = RanochaFluxECandKEP()

fnum = fvol
#fnum = SuliciuFlux()
#fnum = LocalLaxFriedrichsFlux()

p = 2
Nx = 25
Ny = 25

meshx = UniformPeriodicMesh1D(xmin, xmax, Nx)
meshy = UniformPeriodicMesh1D(ymin, ymax, Ny)
basis = LobattoLegendre(p)

tspan = (0., 1.)
semidisc = UniformPeriodicFluxDiffDisc2D(balance_law, meshx, meshy, basis, fvol, fnum, true)
ode = semidiscretise(semidisc, u₀, tspan)

@time sol = solve(ode, SSPRK33(), dt=1/((2p+1)*max(Nx,Ny)), save_everystep=false);

In [ ]:
serial  : 17.284878 seconds ( 73.10 M allocations:  2.101 GiB, 2.04% gc time)
parallel: 51.360241 seconds (690.50 M allocations: 11.266 GiB, 9.98% gc time)

In [ ]:
using BenchmarkTools

u = copy(ode.u0)
du = similar(u)
Pp1 = p+1
jacx = 2 / semidisc.meshx.Δx
jacy = 2 / semidisc.meshy.Δx

u1 = first(u)
u2 = last(u)
direction = Val{:x}()
t = tspan[1]


usethreads = Val{false}()
semidisc = UniformPeriodicFluxDiffDisc2D(balance_law, meshx, meshy, basis, fvol, fnum, false)
display(usethreads)
#display(@benchmark HCL.add_flux_differences_inner_loop!(du,u,balance_law,fvol,Nx,Ny,Pp1,basis.D,jacx,jacy,usethreads))
#@code_warntype HCL.add_flux_differences_inner_loop!(du,u,balance_law,fvol,Nx,Ny,Pp1,basis.D,jacx,jacy,usethreads)
display(@benchmark HCL.add_flux_differences!(du, u, semidisc))
display(@benchmark HCL.add_numerical_fluxes!(du, u, semidisc))
display(@benchmark semidisc(t, u, du))

usethreads = Val{true}()
semidisc = UniformPeriodicFluxDiffDisc2D(balance_law, meshx, meshy, basis, fvol, fnum, true)
display(usethreads)
#display(@benchmark HCL.add_flux_differences_inner_loop!(du,u,balance_law,fvol,Nx,Ny,Pp1,basis.D,jacx,jacy,usethreads))
#@code_warntype HCL.add_flux_differences_inner_loop!(du,u,balance_law,fvol,Nx,Ny,Pp1,basis.D,jacx,jacy,usethreads)
display(@benchmark HCL.add_flux_differences!(du, u, semidisc))
display(@benchmark HCL.add_numerical_fluxes!(du, u, semidisc))
display(@benchmark semidisc(t, u, du))


In [ ]:
using LaTeXStrings, Plots; pyplot()

npoints = 10p+3

xplot, yplot, uplot = evaluate_coefficients(sol[1], meshx, meshy, basis, npoints)
heatmap((xplot, yplot, uplot, balance_law), cbar=true) |> display

xplot, yplot, uplot = evaluate_coefficients(sol[end], meshx, meshy, basis, npoints)
heatmap((xplot, yplot, uplot, balance_law), cbar=true) |> display

# Euler

In [ ]:
import HyperbolicDiffEq; #reload("HyperbolicDiffEq")
using OrdinaryDiffEq

using Plots; pyplot()
using LaTeXStrings

HCL=HyperbolicDiffEq
LobattoLegendre = HCL.LobattoLegendre; UniformPeriodicMesh1D = HCL.UniformPeriodicMesh1D
compute_coefficients = HCL.compute_coefficients; evaluate_coefficients = HCL.evaluate_coefficients
semidiscretise = HCL.semidiscretise; UniformPeriodicFluxDiffDisc2D = HCL.UniformPeriodicFluxDiffDisc2D
EnergyConservativeFlux = HCL.EnergyConservativeFlux; GodunovFLux = HCL.godunov
EulerVar2D = HCL.EulerVar2D; conserved_variables = HCL.conserved_variables; SuliciuFlux=HCL.SuliciuFlux
LocalLaxFriedrichsFlux = HCL.LocalLaxFriedrichsFlux; ChandrashekarFluxEC=HCL.ChandrashekarFluxEC
IsmailRoeFluxEC=HCL.IsmailRoeFluxEC; CentralFlux=HCL.CentralFlux; RanochaFluxECandKEP=HCL.RanochaFluxECandKEP
MorinishiFlux=HCL.MorinishiFlux; DucrosEtAlFlux=HCL.DucrosEtAlFlux; KennedyGruberFlux=HCL.KennedyGruberFlux
PirozzoliFlux=HCL.PirozzoliFlux
integrate=HCL.integrate; kinetic_energy=HCL.kinetic_energy; entropy=HCL.entropy

const γ = 1.4
balance_law = HCL.Euler{Float64,2}(γ)

ϱ₀(x,y)  = 1.
vx₀(x,y) = sin(x)*cos(y)
vy₀(x,y) = -cos(x)*sin(y)
p₀(x,y)  = 100/γ + (cos(2x)+cos(2y))*3/16

u₀(x,y) = conserved_variables(ϱ₀(x,y), vx₀(x,y), vy₀(x,y), p₀(x,y), balance_law)
xmin=0.; xmax=2π; ymin=0.; ymax=2π

#fvol = CentralFlux()
#fvol = MorinishiFlux()
#fvol = DucrosEtAlFlux()
#fvol = KennedyGruberFlux()
fvol = PirozzoliFlux()
#fvol = EnergyConservativeFlux()
#fvol = ChandrashekarFluxEC()
#fvol = IsmailRoeFluxEC()
#fvol = RanochaFluxECandKEP()

fnum = fvol
#fnum = SuliciuFlux()
#fnum = LocalLaxFriedrichsFlux()

p = 1
Nx = 25
Ny = 25

meshx = UniformPeriodicMesh1D(xmin, xmax, Nx)
meshy = UniformPeriodicMesh1D(ymin, ymax, Ny)
basis = LobattoLegendre(p)

tspan = (0., 20.)
semidisc = UniformPeriodicFluxDiffDisc2D(balance_law, meshx, meshy, basis, fvol, fnum, true)
ode = semidiscretise(semidisc, u₀, tspan)

#solve(ode, SSPRK104(), dt=tspan[end]-tspan[1], save_everystep=false)
@time sol = solve(ode, SSPRK104(), dt=1/((2p+1)*max(Nx,Ny)), save_everystep=true);


In [ ]:
idx2ekin(idx) = integrate(u->kinetic_energy(u,balance_law), sol[idx], meshx, meshy, basis)
idx2entr(idx) = integrate(u->entropy(u,balance_law), sol[idx], meshx, meshy, basis)

ekin1 = idx2ekin(1)
entr1 = idx2entr(1)

fig_ekin = plot(sol.t, (map(idx2ekin, eachindex(sol))-ekin1)/abs(ekin1), label="", xguide=L"t", yguide="Rel. Kinetic Energy Difference")
fig_entr = plot(sol.t, (map(idx2entr, eachindex(sol))-entr1)/abs(entr1), label="", xguide=L"t", yguide="Rel. Entropy Difference")

plot(fig_ekin, fig_entr)

In [ ]:
@show integrate(sol[1], meshx, meshy, basis)
@show integrate(sol[end], meshx, meshy, basis)

@show integrate(u->kinetic_energy(u,balance_law), sol[1], meshx, meshy, basis)
@show integrate(u->kinetic_energy(u,balance_law), sol[end], meshx, meshy, basis)

@show integrate(u->entropy(u,balance_law), sol[1], meshx, meshy, basis)
@show integrate(u->entropy(u,balance_law), sol[end], meshx, meshy, basis)

In [ ]:
npoints = 10p+3


xplot, yplot, uplot = evaluate_coefficients(sol[1], meshx, meshy, basis, npoints)
heatmap((xplot, yplot, uplot, balance_law), cbar=true) |> display

xplot, yplot, uplot = evaluate_coefficients(sol[end], meshx, meshy, basis, npoints)
heatmap((xplot, yplot, uplot, balance_law), cbar=true) |> display


# Test Burgers'

In [ ]:
import HyperbolicDiffEq; reload("HyperbolicDiffEq")
using OrdinaryDiffEq

using Plots; pyplot()
using LaTeXStrings

HCL=HyperbolicDiffEq
LobattoLegendre = HCL.LobattoLegendre; UniformPeriodicMesh1D = HCL.UniformPeriodicMesh1D
compute_coefficients = HCL.compute_coefficients; evaluate_coefficients = HCL.evaluate_coefficients
semidiscretise = HCL.semidiscretise; UniformPeriodicFluxDiffDisc2D = HCL.UniformPeriodicFluxDiffDisc2D
EnergyConservativeFlux = HCL.EnergyConservativeFlux; GodunovFLux = HCL.godunov


balance_law = HCL.Burgers{Float64,2}()
u₀(x,y) = sinpi(x); xmin=0.; xmax=2.; ymin=-1.; ymax=1.

fvol = EnergyConservativeFlux()
fnum = GodunovFLux

p = 1
Nx = 50
Ny = 50

meshx = UniformPeriodicMesh1D(xmin, xmax, Nx)
meshy = UniformPeriodicMesh1D(ymin, ymax, Ny)
basis = LobattoLegendre(p)

tspan = (0., 4.)
semidisc = UniformPeriodicFluxDiffDisc2D(balance_law, meshx, meshy, basis, fvol, fnum, true)
ode = semidiscretise(semidisc, u₀, tspan)

solve(ode, SSPRK104(), dt=tspan[end]-tspan[1], save_everystep=false)
@time sol = solve(ode, SSPRK104(), dt=1./((2p+1)*max(Nx,Ny)), save_everystep=false);


In [ ]:
npoints = 10p+3

xplot, yplot, uplot = evaluate_coefficients(sol[1], meshx, meshy, basis, npoints)
heatmap(xplot, yplot, uplot) |> display

xplot, yplot, uplot = evaluate_coefficients(sol[end], meshx, meshy, basis, npoints)
heatmap(xplot, yplot, uplot) |> display

xplot, yplot, uplot = evaluate_coefficients(sol[end], meshx, meshy, basis, npoints)
plot(xplot, uplot[npoints÷2+1,:]) |> display

In [ ]:
using BenchmarkTools

reload("HyperbolicDiffEq")

u = copy(ode.u0)
du = zeros(u)
jacx = 2 / semidisc.meshx.Δx
jacy = 2 / semidisc.meshy.Δx
direction = Val{:x}()

#usethreads = Val{true}()
#usethreads = Val{false}()

#@benchmark HCL.add_flux_differences!(du, u, semidisc)
@benchmark HCL.add_numerical_fluxes!(du, u, semidisc)

#@benchmark fvol(1., 6., balance_law, direction)
#@code_native fvol(1., 6., balance_law, direction)
#@benchmark fnum(1., 6., balance_law, direction)
#@code_native fnum(1., 6., balance_law, direction)

#@code_warntype HCL.add_flux_differences!(du, u, semidisc, Val{true}())
#@code_warntype HCL.add_flux_differences_inner_loop!(du, u, balance_law, fvol, Nx, Ny, p+1, basis.D, jacx, jacy, Val{false}())

# Test Plotting

In [ ]:
import HyperbolicDiffEq; reload("HyperbolicDiffEq")

using Plots; pyplot()
using LaTeXStrings

HCL=HyperbolicDiffEq
LobattoLegendre = HCL.LobattoLegendre; UniformPeriodicMesh1D = HCL.UniformPeriodicMesh1D
compute_coefficients = HCL.compute_coefficients; evaluate_coefficients = HCL.evaluate_coefficients

u₀(x,y) = sinpi(x)*sinpi(y); xmin=-1.; xmax=1.; ymin=-1.; ymax=1.
#u₀(x,y) = (3x + y ^ 2) * abs(sin(x) + cos(y)); xmin=0.; xmax=20.; ymin=-0.; ymax=10.


p = 9
Nx = 10
Ny = 10

meshx = UniformPeriodicMesh1D(xmin, xmax, Nx)
meshy = UniformPeriodicMesh1D(ymin, ymax, Ny)
basis = LobattoLegendre(p)

u0 = compute_coefficients(u₀, meshx, meshy, basis)
xplot, yplot, uplot = evaluate_coefficients(u0, meshx, meshy, basis)


#fig = plot(xguide=L"x", yguide=L"y")
display("contour")
contour(xplot, yplot, uplot) |> display
display("contour(..., fill=true)")
contour(xplot, yplot, uplot, fill=true) |> display
display("heatmap")
heatmap(xplot, yplot, uplot) |> display
display("surface")
surface(xplot, yplot, uplot) |> display
display("contour3d")
contour3d(xplot, yplot, uplot) |> display
display("wireframe")
wireframe(xplot, yplot, uplot) |> display
display("wireframe(..., contours=true)")
wireframe(xplot, yplot, uplot, contours=true) |> display
